<a href="https://colab.research.google.com/github/Shreyash007/deep_learning_course/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 1 CS6910**
Shreyash Gadgil (ED22S016)

In [1]:
!pip install --upgrade wandb
!wandb login aa5afea12b4fda1e7f8310b597eb17c73d1176d2 #my API key for wandb login 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 4.6 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=ba7aad53d0f34f4d2d92071a97b1d268879e504f14d92f4409b50337003e950c
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Import libraries

In [2]:
from keras.datasets import fashion_mnist
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import wandb
from tqdm import tqdm

# **Q1 Solution**
Labelling dataset

In [3]:
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
#importing dataset(add this in show_images function as variable)

#checking size of training and test dataset
print("Training dataset shape, X=",X_train.shape,", Y=",Y_train.shape)
print("Test dataset shape, X=",X_test.shape,", Y=",Y_test.shape)

n_img_classes=len(np.unique(Y_train)) #to find out number of unique images 
img_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] #taken from keras documentation


def show_images(n_img_classes,img_labels,X,Y):

   setImgs = []
   setLabels=[]

   for i in range(len(X)):
     if len(setImgs)==n_img_classes:
       break
     if img_labels[Y[i]] not in setLabels:
      setImgs.append(X[i])
      setLabels.append(img_labels[Y[i]])

   run=wandb.init(project='CS-6910 A1',entity='shreyashgadgil007',reinit=True)
   run.log({"Sample from each class":list(wandb.Image(img,caption= caption) for img,caption in zip(setImgs,setLabels))})
   run.finish()

#--------------------------------------------------------------------------Q1 SOLUTION-----------------------------------------------------------------------------------------------
#show_images(n_img_classes,img_labels,X_test,Y_test)


4422102/4422102 [==============================] - 0s 0us/step
Training dataset shape, X= (60000, 28, 28) , Y= (60000,)
Test dataset shape, X= (10000, 28, 28) , Y= (10000,)


# **Preprocessing dataset for operations**
1. Creating training,test,validation split for dataset
2. Reshaping image matrix and normalizing pixel values
3. One hot encoding classifications


In [4]:
def dataset_preprocess():
  #change the code below to accept different dataset
  (X_1, Y_1), (X_test, Y_test) = fashion_mnist.load_data()
  
  #importing dataset again and normalizing
  X_1 = X_1.reshape(X_1.shape[0],-1)/255.0
  X_test = X_test.reshape(X_test.shape[0],-1)/255.0

  #training and validation split as specified in the question 10%
  X_train, X_val, Y_train, Y_val= train_test_split(X_1,Y_1,test_size=0.1,random_state=0)
  
  #one hot encoding
  Y_train_encoded=np.zeros((Y_train.shape[0],10))
  for i in range(len(Y_train)):
    Y_train_encoded[i][Y_train[i]]=1

  Y_val_encoded=np.zeros((Y_val.shape[0],10))
  for i in range(len(Y_val)):
    Y_val_encoded[i][Y_val[i]]=1

  Y_test_encoded=np.zeros((Y_test.shape[0],10))
  for i in range(len(Y_test)):
    Y_test_encoded[i][Y_test[i]]=1

  return X_train.T,X_test.T,X_val.T,Y_train.T,Y_val.T,Y_test.T,Y_train_encoded.T,Y_val_encoded.T,Y_test_encoded.T

#Below variables will be used in the entire code
X_train,X_test,X_val,Y_train,Y_val,Y_test,Y_train_encoded,Y_val_encoded,Y_test_encoded=dataset_preprocess()
print(Y_val_encoded[0])

[1. 0. 0. ... 0. 0. 0.]


# **Helper functions**
1. Declared activation functions
2. Declared initialisation functions
3. Declared a function to initialize weights and biases

In [25]:
#---------------------------------------------------------------ACTIVATION FUNCTIONS AND THEIR GRADIENTS-------------------------------------------------------------------------------
def relu(X):
  return np.maximum(0,X)

def grad_relu(X):
  return X>0

def sigmoid(X):
  return 1/(1+np.exp(-X))

def grad_sigmoid(X):
  return (sigmoid(X))*(1-sigmoid(X))

def tanh(X):
  return (np.exp(X)-np.exp(-X))/(np.exp(X)+np.exp(-X))

def grad_tanh(X):
  return 1-(tanh(X))**2

def softmax(X):
  # Calculates the softmax function
  e_X = np.exp(X - np.max(X, axis = 0))
  return e_X / e_X.sum(axis = 0)

def softmax_cross_entropy(Y,Y_pred):
  return -(Y - Y_pred) 

#---------------------------------------------------------------------------LOSS FUNCTION---------------------------------------------------------------------------------------------
def cross_entropy():
  return

def squared_error_loss():
  return

#----------------------------------------------------------------------INITIALISATION FUNCTIONS---------------------------------------------------------------------------------------
def random_initialisation(shape):
  # Initialising a random matrix with given dimensions (shape) as tuple
  np.random.seed(0)
  return np.random.randn(*shape)*0.5 #multiplied by 0.5 to have smaller values, to have better initialisation weights

def xavier_initialisation(shape):
    # Calculate the number of inputs and outputs
    n_in = shape[0]
    n_out = shape[1]    
    # Calculate the variance of the normal distribution
    variance = 2.0 / (n_in + n_out)
    # Initialize the weights with random values drawn from the normal distribution
    np.random.seed(0)
    weights = np.random.randn(n_in, n_out) * np.sqrt(variance)*2.0
    
    return weights

#------------------------------------------------------------------INITIALIZING WEIGHTS AND BIASES------------------------------------------------------------------------------------
def initialize_w_b(input_layer,hidden_layer,output_layer,init):
  weights=[]
  biases=[]
  layers=[input_layer]+hidden_layer+[output_layer]
  for i in range(len(hidden_layer)+1):
     if init=='random':
       weights.append(random_initialisation((layers[i+1],layers[i])))
     if init=='xavier':
       weights.append(xavier_initialisation((layers[i+1],layers[i])))
     biases.append(np.random.randn(layers[i+1],1))
  return weights, biases

'''
#testing weights and biases output(used for debugging shape size of matrix) 
output_layer = 10
input_layer = X_test.shape[1]
print(X_test.shape)
hidden_layer = [64, 64]
weights, biases = initialize_w_b(input_layer, hidden_layer, output_layer, 'xavier')
for i in range(len(weights)):
    print(weights[i].shape)
    
for i in range(len(biases)):
    print(biases[i].shape)

weights2, biases2 = initialize_w_b(input_layer, hidden_layer, output_layer, 'random')
for i in range(len(weights2)):
    print(weights2[i].shape)
    
for i in range(len(biases2)):
    print(biases2[i].shape)
'''


"\n#testing weights and biases output(used for debugging shape size of matrix) \noutput_layer = 10\ninput_layer = X_test.shape[1]\nprint(X_test.shape)\nhidden_layer = [64, 64]\nweights, biases = initialize_w_b(input_layer, hidden_layer, output_layer, 'xavier')\nfor i in range(len(weights)):\n    print(weights[i].shape)\n    \nfor i in range(len(biases)):\n    print(biases[i].shape)\n\nweights2, biases2 = initialize_w_b(input_layer, hidden_layer, output_layer, 'random')\nfor i in range(len(weights2)):\n    print(weights2[i].shape)\n    \nfor i in range(len(biases2)):\n    print(biases2[i].shape)\n"

# **Neural Network**
1. With forward propagation
2. Backward propagation with different optimizers
3. A train function, which fits and trains the given set

In [10]:
class NeuralNet():

  def __init__(self,input_layer,hidden_layer,output_layer,initialisation_func,act_function,loss_func):
     self.input_layer=input_layer
     self.hidden_layer=hidden_layer
     self.output_layer=output_layer
     self.initialisation_func=initialisation_func
     self.act_function=act_function
     self.loss_func=loss_func
     self.weights,self.biases = initialize_w_b(self.input_layer,self.hidden_layer,self.output_layer,self.initialisation_func)
     self.layer_size=len(self.hidden_layer)

  def forward_propagation(self,X):
     #pre-activation
     self.a=[]
     #post-activation
     self.h=[]
     l=0
     
     #pre-activation and post-activation for input layer and first hidden layer
     self.a.append((self.weights[l]@X)+self.biases[l])#WX+b
     if self.act_function=='sigmoid':
       self.h.append(sigmoid(self.a[l]))
     elif self.act_function=='relu':
       self.h.append(relu(self.a[l]))
     elif self.act_function=='tanh':
       self.h.append(tanh(self.a[l]))
     
     #pre-activation and post-activation between hidden layers
     for l in range(1,self.layer_size):
       self.a.append((self.weights[l]@self.h[l-1])+self.biases[l]) #W.X+b
       if self.act_function=='sigmoid':
        self.h.append(sigmoid(self.a[l]))
       elif self.act_function=='relu':
        self.h.append(relu(self.a[l]))
       elif self.act_function=='tanh':
        self.h.append(tanh(self.a[l]))
       
     #pre-activation and post-activation between last hidden layer and output layer
     l=self.layer_size 
     self.a.append((self.weights[l]@self.h[l-1])+self.biases[l]) #WX+b
     self.h.append(softmax(self.a[l]))
     
     #we only need the output of the end layer
     return self.h[-1]

#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------Q3 BACK PROPPAGATION FRAMEWORK WITH OPTIMIZATION FUNCTIONS-----------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  
  def back_propagation(self,X,Y):

        g_a  = [0]*(self.layer_size+1)
        g_h  = [0]*(self.layer_size+1)
        g_w  = [0]*(len(self.weights))
        g_b  = [0]*(len(self.biases))
        #Due to error accumulation the gradients values increases, so we average the value with batch size 
        batch_size = X.shape[1] 

        for k in reversed(range(self.layer_size+1)):
            #calculating loss function for weights and biases at output
            if k == self.layer_size:
              if self.loss_func == 'cross_entropy':
                  g_a[k] = self.h[k]  - Y  
              elif self.loss_func == 'square_loss': 
                  g_a[k] = (self.h[k] - Y) * self.h[k] * (1 - self.h[k]) 
            
            #calculating gradients for hidden layers     
            else:
                g_h[k] = (1/batch_size)*((self.weights[k+1].T)@(g_a[k+1]))
                if self.act_function == 'sigmoid':
                  g_a[k] = (1/batch_size)*((g_h[k])*(grad_sigmoid(self.a[k])))#Here we use '*' operator for elementwise matrix multiplication
                elif self.act_function == 'tanh':
                  g_a[k] = (1/batch_size)*((g_h[k])*(grad_tanh(self.a[k])))#Here we use '*' operator for elementwise matrix multiplication
                elif self.act_function == 'relu':
                  g_a[k] = (1/batch_size)*((g_h[k])*(grad_relu(self.a[k])))#Here we use '*' operator for elementwise matrix multiplication
                
            #calculating gradients of weights 
            if k == 0:
                g_w[k] = (1/batch_size)*((g_a[k])@(X.T)) 
            else:
                g_w[k] = (1/batch_size)*((g_a[k])@(self.h[k-1].T))
            
            #calculating gradients of biases
            g_b[k]  = (1/batch_size)*np.sum(g_a[k], axis=1, keepdims = True)
        return g_w,g_b
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#-----------------------------------------------------STOCHASTIC GRADIENT DESCENT AND OTHER OPTIMIZERS-------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  
  def stochastic_gradient_descent(self,W,b,g_w,g_b,lr,lambd):
      #Weights=Weights-(learning rate)*(grad weights)-(learning rate*lambda)*(Weights)(this is weight decay with L2 regularization)
      W=W-np.multiply(lr,g_w)-np.multiply(lr*lambd,W)

      #biases=biases-(learning rate)*(grad biases)
      b=b-np.multiply(lr,g_b)
      return W,b
  
  def momentum_based_gradient_descent(self,W,b,g_w,g_b, u_w_i, u_b_i, lr, lambd, gamma):
      #u_t=(beta)*(u_t-1)+(grad weight)[u_0 is initialised as zero]
      u_w=np.multiply(gamma,u_w_i)+g_w
      #weights=weights-(lr)*(updated_weights)-(weight decay L2 regularization)
      W=W-np.multiply(lr,u_w)-np.multiply(lr*lambd,W)
      
      #similarly for biases but without weight decay term
      u_b=np.multiply(gamma,u_b_i)+g_b
      b=b-np.multiply(lr,u_b)
      return W,b,u_w,u_b
  
  def nesterov_accelerated_gradient_descent(self,W,b,g_w,g_b,lr,lambd,gamma,u_w_i, u_b_i,b_input,b_output):
      w_t=W
      b_t=b
      #here we make changes to global variables as we need to update the weights(look ahead) for calculating gradients
      self.weights = self.weights - np.multiply(gamma,u_w_i)
      self.biases = self.biases - np.multiply(gamma,u_b_i)
      output =  self.forward_propagation(b_input)
      #in this step, gradients are recalculated in global variables with updated weight values 
      g_weights,g_biases = self.back_propagation(b_input,b_output)

      #u_t=(gamma)*(u_(t-1))+gradient of(w_t-(gamma)*(u_(t-1)))
      u_w=np.multiply(gamma,u_w_i)+g_weights
      #weights=weights-(lr)*(updated_weights)-(weight decay L2 regularization)
      w_t = w_t - u_w - np.multiply(lr*lambd,w_t)

      #similarly for biases but without weight decay term
      u_b=np.multiply(gamma,u_b_i)+g_b
      b=b-np.multiply(lr,u_b) 

      return w_t,b,u_w,u_b
  
  def rmsprop(self,W,b,g_w,g_b,lr,lambd,beta,eps,vw,vb):
      #V_t= (beta)*(V_t-1)+(1-beta)*(grad weight)^2 
      vw = np.multiply(vw,beta) + np.multiply(1-beta,np.power(g_w,2))           
      #similarly for biases
      vb = np.multiply(vb,beta) + np.multiply(1-beta,np.power(g_b,2))
      
      #w_t= (w_t-1)-((lr)/(V_t+eps)^(1/2))*(grad weight)-(L2 regularization weight decay)
      W = W - np.multiply(g_w,lr/np.power(vw+eps,1/2))- np.multiply(lr*lambd,W)
      #similarly for biases
      b = b - np.multiply(g_b,lr/np.power(vb+eps,1/2))
      return W,b,vw,vb
  
  def adam(self,W,b,g_w,g_b,beta1,beta2,lr ,m_t_i ,v_t_i ,m_b_i ,v_b_i,eps,i,lambd):
      
      m_t = np.multiply(beta1,m_t_i) + np.multiply(1-beta1,g_w)
      v_t = np.multiply(beta2,v_t_i) + np.multiply(1-beta2,np.power(g_w,2))
      m_b = np.multiply(beta1,m_b_i) + np.multiply(1-beta1,g_b)
      v_b = np.multiply(beta2,v_b_i) + np.multiply(1-beta2,np.power(g_b,2))
                
      #normalization of moment          
      m_hat_w = m_t/(1 - np.power(beta1,i+1))
      m_hat_b = m_b/(1 - np.power(beta1,i+1))

      #normalization          
      v_hat_w = v_t/(1 - np.power(beta2,i+1))
      v_hat_b = v_b/(1 - np.power(beta2,i+1))
      
      W = W - ((lr / np.power(v_hat_w + eps, 1/2)) * m_hat_w) - np.multiply(lr*lambd,W)
      
      b = b - ((lr / np.power(v_hat_b + eps, 1/2)) * m_hat_b)
      return W,b,m_t,v_t,m_b,v_b
  
  def nadam(self,W,b,g_w,g_b,beta1,beta2,lr ,m_t_i ,v_t_i ,m_b_i ,v_b_i,eps,i,lambd):
      m_t =  np.multiply(beta1,m_t_i) + np.multiply(1 - beta1,g_w)
      v_t =  np.multiply(beta2,v_t_i) + np.multiply(1 - beta2,np.power(g_w, 2))

      m_b =  np.multiply(beta1,m_b_i) + np.multiply(1 - beta1,g_b)
      v_b =  np.multiply(beta2,v_b_i) + np.multiply(1 - beta2,np.power(g_b, 2))
                
      m_hat_w = m_t / (1 - np.power(beta1, i+1)) 
      v_hat_t = v_t / (1 - np.power(beta2, i+1))

      m_hat_b = m_b / (1 - np.power(beta1, i+1)) 
      v_hat_b = v_b / (1 - np.power(beta2, i+1))
  
      a1 = (1-beta1)/(1-np.power(beta1,i+1))
      update_w = np.multiply(lr/(np.power(v_hat_t + eps,1/2)),(np.multiply(a1,g_w) + np.multiply(beta1,m_hat_w)))
      update_b = np.multiply(lr/(np.power(v_hat_b + eps,1/2)),(np.multiply(a1,g_b)+np.multiply(beta1,m_hat_b) ))
      W = W - update_w - np.multiply(lr*lambd,W)
      b = b - update_b    
      return W,b,m_t,v_t,m_b,v_b
  


  def predict(self, X,y ):
      output =  self.forward_propagation(X)
      out_class=(np.argmax(output,axis=0))
      accuracy = round(accuracy_score(y, out_class) * 100, 3)
      return accuracy , out_class

  def train(self,X_train,y_train,X_val ,y_val ,learning_rate,epochs, optimiser='gd',batch_size = 64,lambd=0.0005,WandB=False): 

      update_w = np.zeros(np.array(self.weights).shape)
      update_b = np.zeros(np.array(self.biases).shape)
      update_w_i = np.zeros(np.array(self.weights).shape)
      update_b_i = np.zeros(np.array(self.biases).shape)
      vw_i = 0.0
      vb_i = 0.0
      m_t_i=0
      v_t_i=0
      m_b_i=0
      v_b_i=0
      eps = 1e-8
      gamma = 0.9
      beta = 0.999
      beta1 = 0.9
      beta2 = 0.999
      m_t, v_t, m_hat_w, v_hat_w, m_b,v_b,m_hat_b,v_hat_b = 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0 

      train_accuracy, val_accuracy, training_loss ,validation_loss = [] ,[] ,[] ,[]
      for i in tqdm(range(epochs)):
        for batch in range(0, X_train.shape[1], batch_size):

          batch_images =  X_train[:,batch:batch+batch_size]
          batch_output =  Y_train_encoded[:,batch:batch+batch_size]
          output =  self.forward_propagation(batch_images)
          g_weights,g_biases = self.back_propagation(batch_images,batch_output)
          if optimiser == 'gd':
              self.weights,self.biases=self.stochastic_gradient_descent(self.weights,self.biases,g_weights,g_biases,learning_rate,lambd)
          
          if optimiser == 'mgd':
              self.weights,self.biases,update_w,update_b=self.momentum_based_gradient_descent(self.weights,self.biases,g_weights,g_biases,update_w_i,update_b_i,learning_rate,lambd,gamma)
              update_w_i = update_w
              update_b_i = update_b

          if optimiser == 'ngd':
              w_t,b_t,update_w,update_b=self.nesterov_accelerated_gradient_descent(self.weights,self.biases,g_weights,g_biases,learning_rate,lambd,gamma,update_w_i,update_b_i,batch_images,batch_output)
              self.weights = w_t
              self.biases = b_t
              update_w_i = update_w
              update_b_i = update_b


          if optimiser == 'rmsprop': 
              self.weights,self.biases,vw,vb= self.rmsprop(self.weights,self.biases,g_weights,g_biases,learning_rate,lambd,beta,eps,vw_i,vb_i)
              vw_i=vw
              vb_i=vb

          if optimiser == 'adam':
              self.weights,self.biases,m_t,v_t,m_b,v_b=self.adam(self.weights,self.biases,g_weights,g_biases,beta1,beta2,learning_rate, m_t_i, v_t_i, m_b_i, v_b_i,eps,i,lambd)
              m_t_i=m_t
              v_t_i=v_t
              m_b_i=m_b
              v_b_i=v_b

          
          if optimiser == 'nadam':
              self.weights,self.biases,m_t,v_t,m_b,v_b=self.nadam(self.weights,self.biases,g_weights,g_biases,beta1,beta2,learning_rate, m_t_i,v_t_i, m_b_i,v_b_i, eps, i, lambd)
              m_t_i=m_t
              v_t_i=v_t
              m_b_i=m_b
              v_b_i=v_b                         


        #Training loss for full dataset
        predicted_train = self.forward_propagation(X_train)
        predicted_train_label=(np.argmax(predicted_train,axis=0)) 
        acc1 = 100*np.sum(predicted_train_label==y_train)/predicted_train.shape[1]
        train_accuracy.append(acc1)


        predicted_val = self.forward_propagation(X_val)
        predicted_val_label=(np.argmax(predicted_val,axis=0))
        acc2 = 100*np.sum(predicted_val_label==y_val)/predicted_val.shape[1]   
        val_accuracy.append(acc2)

        a =self.weights[1:len(self.hidden_layer)]
        b = np.sum([(np.sum((a[i]**2).reshape(1,-1))) for i in range(len(a))]) 
        if self.loss_func == 'cross_entropy':
          train_loss = (-np.sum(np.multiply(Y_train_encoded,np.log(predicted_train)))+((lambd/2.)*b))/y_train.shape[0]
          val_loss = (-np.sum(np.multiply(Y_val_encoded,np.log(predicted_val)))+((lambd/2.)*b))/y_val.shape[0]

        elif self.loss_func == 'square_loss':

          val_loss =((1/2) * np.sum((Y_val_encoded- predicted_val)**2))/y_val.shape[0]
          train_loss = (1/2) * np.sum((Y_train_encoded - predicted_train)**2)/y_train.shape[0]
        training_loss.append(train_loss)
        validation_loss.append(val_loss)
          

        print('Epoch {} : training_accuracy = {:.2f}, training_loss = {:.5f},Validation accuracy = {:.2f},Validation loss = {:.5f}'.format(i+1,acc1,train_loss, acc2,val_loss))
        if WandB==True:
          wandb.log({"val_accuracy": acc2,"accuracy": acc1,"steps":epochs,"train_loss":train_loss,"val_loss":val_loss},)
        
      
      return train_accuracy,val_accuracy,training_loss, validation_loss



Q2 Forward propagation solved

In [31]:
n_class = 10

model = NeuralNet(28*28,[128,128,128],n_class, initialisation_func = 'xavier', act_function='tanh', loss_func = 'cross_entropy')
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
acc1,acc2,train_loss, val_loss= model.train(X_train,Y_train,X_val ,Y_val ,learning_rate = 0.01,epochs=5, optimiser='adam',batch_size =32,lambd=0.0005)


 20%|██        | 1/5 [00:15<01:03, 15.86s/it]

Epoch 1 : training_accuracy = 80.00, training_loss = 0.54152,Validation accuracy = 80.10,Validation loss = 0.55204


 40%|████      | 2/5 [00:32<00:48, 16.10s/it]

Epoch 2 : training_accuracy = 81.66, training_loss = 0.49802,Validation accuracy = 81.00,Validation loss = 0.51371


 60%|██████    | 3/5 [00:47<00:31, 15.80s/it]

Epoch 3 : training_accuracy = 82.71, training_loss = 0.47132,Validation accuracy = 82.22,Validation loss = 0.49101


 80%|████████  | 4/5 [01:02<00:15, 15.47s/it]

Epoch 4 : training_accuracy = 83.18, training_loss = 0.45617,Validation accuracy = 82.75,Validation loss = 0.47944


100%|██████████| 5/5 [01:17<00:00, 15.53s/it]

Epoch 5 : training_accuracy = 83.62, training_loss = 0.44449,Validation accuracy = 82.97,Validation loss = 0.47084


In [ ]:
q1    = NeuralNet(X_train.shape[0],[64,64,64], 10,'xavier','sigmoid','cross_entropy')
A=q1.forward_propagation(X_train)
A=A.T
print(A[0])
print(X_train.shape[1])
print(Y_train[0])
print((Y_train_encoded.T)[0])

In [ ]:
sweep_config = {
    'method': 'bayes', #grid, random,bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'epochs': {
            'values': [15,20,30]
        },
        'learning_rate': {
            'values': [0.001,0.0001]
        },
        'loss_function':{
            'values':['cross_entropy']
        },
        'initilisation':{
            'values':['random','xavier']
        },
        'batch_size':{
            'values':[32,64,128]
        },
        'optimiser': {
            'values': ['gd','mgd','ngd', 'rmsprop','adam','nadam']
        },
        'activation': {
            'values': ['relu','tanh','sigmoid']
        },
        'hidden_layer': {
            'values': [
                       [64,64,64],
                       [64,64,64,64],[64,64,64,64,64],[128,128,128],
                       [128,128,128,128],[128,128,128,128,128],[256,256,256]]
        },
        
        'weight_decay':{
            'values':[0.0,0.0005]
        }
        
    }
}

sweep_id = wandb.sweep(sweep_config, entity='shreyashgadgil007', project="CS-6910 A1")

def train1():
  steps = 10
  # Default values for hyper-parameters we're going to sweep over
  config_defaults = {
      'epochs': 5,
      'input_layer': 784,
      'output_layer': 10,
      'batch_size':64,
      'weight_decay':0.005,
      'learning_rate': 0.0001,
      'hidden_layer':[16,16,16],
      'optimiser':'mgd',
      'activation':'sigmoid',
      'initialisation':'xavier',
      'loss_function':'cross_entropy'
      
  }

  # Initialize a new wandb run
  wandb.init(project='CS-6910 A1', entity='shreyashgadgil007',config=config_defaults)
  wandb.run.name = 'b_s:'+str(wandb.config.batch_size)+',lr:'+ str(wandb.config.learning_rate)+',ep:'+str(wandb.config.epochs)+ ',opt:'+str(wandb.config.optimiser)+ ',hl:'+str(wandb.config.hidden_layer)+ ',act:'+str(wandb.config.activation)+',decay:'+str(wandb.config.weight_decay)+',init:'+str(wandb.config.initialisation)+',loss:'+str(wandb.config.loss_function)

  
  # Config is a variable that holds and saves hyperparameters and inputs
  config = wandb.config
  learning_rate = config.learning_rate
  epochs = int(config.epochs)
  hidden_layer = config.hidden_layer
  activation = config.activation
  optimiser = config.optimiser
  input_layer = config.input_layer
  output_layer = config.output_layer
  batch_size = config.batch_size
  weight_decay = config.weight_decay
  loss_function = config.loss_function
  initialisation = config.initilisation
  # Model training here
  sweep_network    = NeuralNet(input_layer, hidden_layer, output_layer,initialisation,activation,loss_function)
  acc1,acc2,train_loss,val_loss  = sweep_network.train(X_train,Y_train,X_val,Y_val,learning_rate,epochs,optimiser,batch_size,weight_decay)
  # print('Acc',acc2)

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
wandb.agent(sweep_id, function=train1, count=120)
